In [1]:
import pandas as pd
import numpy as np
import config as cfg
import json

# Load py2neo
import py2neo
from py2neo import Graph
from py2neo.matching import *

# Interactive Plotting Libraries
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import plotly.graph_objects as go

# Plotting Widgets
import cufflinks as cf

# Throughput Github Analysis

This is a research project led by PhD Simon Goring.

Different research questions are tried to be answer such as: 

- How do individuals and organizations use GitHub (or other public code repositories) to reference, analyze or reuse data from Data Catalogs?

- Are there clear patterns of use across public repositories?

- Do patterns of use differ by data/disciplinary domain, or do properties of the data resource (presence of an API, online documentation, size of user community) affect patterns of use? 

- Does the data reuse observed here expand our understanding of current modes of data reuse, e.g. those outlined in https://datascience.codata.org/articles/10.5334/dsj-2017-008/ ?

- What are the characteristics and shape of the Earth Science research object network?
- What are major nodes of connectivity?
- What poorly connected islands exist? 
- What is the nature of data reuse in this network?
- What downstream/second order grant products can be identified from this network?

## Current Approach

Categorizing a subset of scraped repos, with pre-defined types, which may be updated iteratively as categorization progresses (education, analysis, archiving, informational).


Using ML techniques, we might be able to classify repos according to type automatically; and could consider classifying according to repository quality/completeness. Repository quality or completeness would be defined by:

- presence/absence/length of readme
- number of commits
- number of contributors

By using neo4j, we can construct and analyze the network graph in order to get:
- Centrality and level of connection
- Identification of small networks/islands within the network
- What databases are highly connected and which are not?
- Use database properties (has API, online search portal, has R/Python package, has user forum . . .)

## Objective of the Notebook

This Notebook is going to be used to created an initial Data Exploratory using Neo4j in order to later on, create a Recommendation System using of graph databases. 

In its initial stages, it might look rough, but this will be improved as it is updated and upgraded.

First, let's connect to Neo4j's graph.

There is a `config.py` script, imported as `cfg` that includes personal credentials to log into the database. A `config_sample.py` script has been included. There, change the words `username` and `password` accordingly to match your own credentials.

The port that neo4j automatically usees is 7687 when working in a local database.

In [2]:
# Connect to Graph
graph = Graph("bolt://localhost:7687", auth=(cfg.neo4j['auth']), bolt=True, password=cfg.neo4j['password'])

In [3]:
graph

Graph('bolt://neo4j@localhost:7687', name='neo4j')

In order to select nodes that are a certain kind, we use the command `match`. 

In order to run queries, you can do `graph.run()` and do the Querie inside quotes. Get data using the verb `.data()`

In [4]:
trial = graph.run("MATCH (n:AGENT) RETURN n LIMIT 10").data()
trial[1]

{'n': Node('AGENT', homepage='https://github.com/throughput-ec/throughputdb/keywordMgmt', name='Keyword synonymy')}

As seen above, the nature of nested dictionaries in lists, will definitely represent a challenge when trying to organized data and functions will be needed to make sure each observation's data is appropriately organized in the corresponding features. To convert a list that should be a dictionary use: `json.loads(list_that_should_be_dictionary)`

### Counting observations

In [5]:
graph.run('MATCH (crt:TYPE {type:"schema:CodeRepository"})\
           MATCH (crt)<-[:isType]-(ocr:OBJECT) \
           RETURN COUNT(DISTINCT ocr)').to_data_frame()

,COUNT(DISTINCT ocr)
0,73563


# EDA to get the right queries

In order to figure out how to create a ML model, we need to extract the correct data from the Throughput database.

We will analyze and graph the following:
- Distribution of references to DBs

- Note 'Earth Science' databases within graph
    - X = DBs; y = # of referenced repos
    - Linked repos (x) by commits (y)

- Note ES commits 
    - Linked repos (x) by # of contributors (y)
    - Linked repos (x) by # of forks (y)

## Getting DataCatalogs and Counts(CodeRepos)

In [6]:
counts = graph.run('''MATCH (k:KEYWORD {keyword: "earth science"})\
MATCH (k)<-[:hasKeyword]-(:ANNOTATION)-[:Body]->(dc:dataCat)\
MATCH (dc)<-[:Target]-(:ANNOTATION)-[:Target]->(cr:codeRepo)\
RETURN DISTINCT properties(dc), count(DISTINCT cr)''').data()

![](img/01_graph.png)

### Example on extracting data

In [7]:
# Extracting ID of Data Catalog
counts[1]['properties(dc)']['id']

'r3d100010867'

In [8]:
# Extracting number of CodeRepos linked to Data Catalog
counts[1]['count(DISTINCT cr)']

8

In [9]:
# Put DataCatalogs ID's and CodeRepo's counts together

helper_dict={'item': [],
            'counts':[]}

for i in range (0, len(counts)-1):
    helper_dict['item'].append(counts[i]['properties(dc)']['id'])
    helper_dict['counts'].append(counts[i]['count(DISTINCT cr)'])

counts_df = pd.DataFrame(helper_dict)
counts_df = counts_df.rename(columns={'item':'dacat', 'counts':'cr_counts'})

## Getting Other MetaData

In [10]:
data = graph.run('''MATCH (k:KEYWORD {keyword: "earth science"})\
MATCH (k)<-[:hasKeyword]-(a1:ANNOTATION)-[:Body]->(dc:dataCat)\
MATCH (dc)<-[:Target]-(a2:ANNOTATION)-[:Target]->(cr:codeRepo)\
RETURN distinct properties(dc), properties(cr)''').data()

In [11]:
dict1 = data[0]
dict1.keys()

dict_keys(['properties(dc)', 'properties(cr)'])

In [12]:
dict1['properties(dc)']['id']

'r3d100010356'

In [13]:
dict1['properties(cr)']['meta']

'{"id": 37471462, "repo": "ramadda", "owner": "donmurray", "name": "donmurray/ramadda", "url": "https://github.com/donmurray/ramadda", "created": "2015-06-15 (14:49:29.000000)", "description": null, "topics": [], "readme": {"readme": {"readme": true, "badges": 0, "headings": 0, "char": 3369}, "license": "Other"}, "commits": {"totalCommits": 5604, "range": ["2015-06-09 (01:08:44.000000)", "2015-06-13 (12:33:09.000000)"], "authors": [null]}, "languages": {"Java": 10267669, "JavaScript": 1088747, "HTML": 958926, "CSS": 384048, "Tcl": 156286, "Shell": 30045, "Batchfile": 653, "Python": 210}, "stars": 0, "forks": 0, "fork": false, "issues": 0, "branches": 1, "watchers": 0, "checkdate": "2020-11-08 (23:59:26.409711)"}'

In [14]:
string = dict1['properties(cr)']['meta'] # this is a string, from here, using find and REGEX, get commits 
string

'{"id": 37471462, "repo": "ramadda", "owner": "donmurray", "name": "donmurray/ramadda", "url": "https://github.com/donmurray/ramadda", "created": "2015-06-15 (14:49:29.000000)", "description": null, "topics": [], "readme": {"readme": {"readme": true, "badges": 0, "headings": 0, "char": 3369}, "license": "Other"}, "commits": {"totalCommits": 5604, "range": ["2015-06-09 (01:08:44.000000)", "2015-06-13 (12:33:09.000000)"], "authors": [null]}, "languages": {"Java": 10267669, "JavaScript": 1088747, "HTML": 958926, "CSS": 384048, "Tcl": 156286, "Shell": 30045, "Batchfile": 653, "Python": 210}, "stars": 0, "forks": 0, "fork": false, "issues": 0, "branches": 1, "watchers": 0, "checkdate": "2020-11-08 (23:59:26.409711)"}'

In [15]:
response = json.loads(string) 
response['forks']

0

In [16]:
response['id']

37471462

## Metadata to DF

In [17]:
helper_dict = None
helper_dict = {'dacat': [],
               'meta':[],
               'cr_item': [],
               'forks':[],
               'commits':[],
               'contributors':[]}

for i in range (0, len(data)-1):
    helper_dict['dacat'].append(data[i]['properties(dc)']['id'])
    try:
        helper_dict['meta'].append(data[i]['properties(cr)']['meta'])
        json_data = json.loads(data[i]['properties(cr)']['meta'])
        helper_data = json_data['id']
        
        # Forks
        forks = json_data['forks']
        helper_dict['cr_item'].append(helper_data)
        helper_dict['forks'].append(forks)
        
        # Commits
        commits = json_data['commits']['totalCommits']
        helper_dict['commits'].append(commits)
        
        # Contributors 
        contributors = json_data['commits']['authors']
        helper_dict['contributors'].append(len(contributors))
        
    # Take care of empty spaces.    
    except KeyError:
        helper_dict['meta'].append("None2")
        helper_dict['cr_item'].append("Missing")
        helper_dict['forks'].append("Missing")
        helper_dict['commits'].append("Missing")
        helper_dict['contributors'].append("Missing")
        

meta_df = pd.DataFrame(helper_dict)
meta_df = meta_df[meta_df['meta'] != "None2"]
meta_df = meta_df[['dacat', 'cr_item', 'forks', 'commits', 'contributors']]
meta_df = meta_df.astype({'cr_item':'str', 'forks': 'int64', 'commits': 'int64', 'contributors': 'int64'})

In [18]:
meta_df.describe()

,forks,commits,contributors
count,328.000000,328.000000,328.000000
mean,5.987805,1158.920732,5.463415
std,29.254757,4694.785140,28.739873
min,0.000000,1.000000,1.000000
25%,0.000000,12.000000,1.000000
50%,0.000000,54.000000,2.000000
75%,2.000000,251.500000,3.000000
max,417.000000,55881.000000,504.000000


### Grouping/Plotting by Data Catalog

In [19]:
grouped_dc = None
grouped_dc = meta_df.groupby('dacat').sum().reset_index()
grouped_dc = grouped_dc.merge(counts_df)
grouped_dc = grouped_dc[['dacat', 'cr_counts', 'forks', 'commits', 'contributors']]

In [20]:
@interact(x=(0,500))
def show_dc_more_than(column=['cr_counts','forks', 'commits', 'contributors'], x = 1):
    grouped_dc
    
    return grouped_dc.loc[grouped_dc[column] > x]

interactive(children=(Dropdown(description='column', options=('cr_counts', 'forks', 'commits', 'contributors')…

In [21]:
@interact
def histogram_plot(x = list(grouped_dc.select_dtypes('object').columns), 
                   y = list(grouped_dc.select_dtypes('int64').columns)[0:]):
    
    # trace
    trace = [go.Bar(x=grouped_dc[x], y=grouped_dc[y])]

    # layout
    layout = go.Layout(
                title = 'Counts plot', # Graph title
                xaxis = dict(title = x.title()), # x-axis label
                yaxis = dict(title = y.title()), # y-axis label
                hovermode ='closest' # handles multiple points landing on the same vertical
    )

    # fig
    fig = go.Figure(trace, layout)
    fig.show()

interactive(children=(Dropdown(description='x', options=('dacat',), value='dacat'), Dropdown(description='y', …

### Grouping/Plotting by Code Repository

In [22]:
grouped_cr = None
grouped_cr = meta_df.groupby('cr_item').sum().reset_index()
grouped_cr['cr_item']=grouped_cr['cr_item']+'cr'

In [23]:
@interact(x=(0,500))
def show_dc_more_than(column=['forks', 'commits', 'contributors'], x = 1):
    grouped_cr
    
    
    return grouped_cr.loc[grouped_cr[column] > x]

interactive(children=(Dropdown(description='column', options=('forks', 'commits', 'contributors'), value='fork…

In [24]:
@interact
def histogram_plot(x = list(grouped_cr.select_dtypes('object').columns), 
                   y = list(grouped_cr.select_dtypes('int64').columns)[0:]):
    
    # trace
    trace = [go.Bar(x=grouped_cr[x], y=grouped_cr[y])]

    # layout
    layout = go.Layout(
                title = 'Counts plot', # Graph title
                xaxis = dict(title = x.title()), # x-axis label
                yaxis = dict(title = y.title()), # y-axis label
                hovermode ='closest' # handles multiple points landing on the same vertical
    )

    # fig
    fig = go.Figure(trace, layout)
    fig.show()

interactive(children=(Dropdown(description='x', options=('cr_item',), value='cr_item'), Dropdown(description='…

## Filtering Surprising Data Points for Analysis

## Extraodinary Repo over 400 Forks

In [25]:
meta_df[meta_df['forks']>300]

,dacat,cr_item,forks,commits,contributors
302,r3d100011758,12745174,417,55881,504


## Data Catalog it belongs to

In [26]:
meta_df[(meta_df['dacat']=='r3d100011758') & (meta_df['forks']>50)]

,dacat,cr_item,forks,commits,contributors
302,r3d100011758,12745174,417,55881,504
314,r3d100011758,33125718,142,12237,3
328,r3d100011758,90807748,79,112,2
